### 1. Objetivos
**Geral**: encontrar RNA de plantas tidos como não codificantes mas que, na verdade, codificam proteínas

**Específico**: Varrer os bancos de dados de proteômica em busca de proteínas candidatas a serem codificadas por mRNAs tidos como não codificantes


### 2. PRoteomcis IDEntifications Database (PRIDE) [link](https://www.ebi.ac.uk/pride/)

Visão geral do PRIDE

In [1]:
from filter_datasets import *
import pandas as pd
import json
import collections
import seaborn as sn
import matplotlib.pyplot as plt
import ppx #A Python interface to proteomics data repositories
from pyteomics import mztab
from ete3 import Tree, NodeStyle, TreeStyle, NodeStyle, faces, AttrFace, CircleFace
from ete3 import NCBITaxa
import glob
import urllib.parse
import urllib.request
import uniprot
import requests
from Bio import SeqIO
from io import StringIO
from pandas_profiling import ProfileReport

In [4]:
# pride_plant.json foi gerado usando a função filterPride para checar o número de projetos com plantas 
with open('pride_plant.json') as pride_plants_file:
    pride_plant = json.load(pride_plants_file)
print(f"Total de projetos com plantas no PRIDE: {len(pride_plant)}")

Total de projetos com plantas no PRIDE: 1464


In [5]:
pd.DataFrame(pride_plant,).T.head()

,name,accession
PRD000037,Chlamydomonas reinhardtii,3055
PRD000044,Arabidopsis thaliana (mouse-ear cress),3702
PRD000051,Zea mays (maize),4577
PRD000084,Arabidopsis thaliana (mouse-ear cress),3702
PRD000096,Arabidopsis thaliana (mouse-ear cress),3702


In [6]:
# Pride_plants_mztab.json foi gerado usando a função get_mztabes. Ele contem os projetos com plantas e com arquivos mztab
with open("pride_plants_mztab.json") as mztabs:
    pride_plants_mztab = json.load(mztabs)
print(f"Total de projetos com arquivos mztab: {len(pride_plants_mztab)}")

Total de projetos com arquivos mztab: 270


Arquivos mztab por espécie

In [7]:
pride_plants_names = [values["name"] for keys,values in pride_plant.items()]
pride_plants_counter = collections.Counter(pride_plants_names)
plant_names = [plants[0] for plants in pride_plants_counter.items()]
plant_quant = [plants[1] for plants in pride_plants_counter.items()]
plant_with_mztab = [] 
for specie in pride_plants_counter.keys():
    all_projects = [projects for projects,values in pride_plant.items() if f"{specie}" == values["name"]]
    mztab_projects = list(set(pride_plants_mztab.keys()) & set(all_projects))
    plant_with_mztab.append(len(mztab_projects))
plants_count_dataframe = pd.DataFrame({
    "Specie":plant_names,
    "Total_projects":plant_quant,
    "Projects_with_mztab":plant_with_mztab
})
plants_count_dataframe.sort_values("Total_projects", ascending = False, inplace = True)


In [10]:
plt.figure(figsize = (12,8))
plot = plt.subplot()
projects = plot.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Total_projects[:20],color = 'b',align='center', alpha = .6)
mztabs = plot.barh(plants_count_dataframe.Specie[:20],plants_count_dataframe.Projects_with_mztab[:20],color = 'g',align='center', alpha = 0.6,)
plot.set_ylabel("Espécie",fontsize = 20)
plot.set_xlabel("Número de projetos",fontsize = 20)
plot.set_title("20 plantas mais comuns (PRIDE)", fontsize = 30)
plot.tick_params(labelsize = 18)
plot.legend((projects,mztabs),("Total de projetos","Projetos com arquivos mztab"),fontsize = 20)
plt.savefig("/home/tiago/documents/lncRNA/figures/fig1_most_commom_plants.png")

### 3. Caminhos possíveis 

1. Os arquivos do PRIDE contem peptídeos que fazem parte de proteínas não caracterizadas e que podem ser mapeadas de volta em regiões codificam RNAs não codificantes nos genomas das espécias alvo.

2. Os arquivos do PRIDE contem peptídeos qua não estão nos proteoma das espécies alvo.


![Fluxograma](figures/fluxograma.jpg)

#### 3.1. Caminho 1



In [11]:
def get_projects(dict_plants = pride_plant, dict_mztab = pride_plants_mztab, specie = None):
    """
    Returns all projecst with mztab files for a give plant specie. Search by name
    parameters:
        dict_plants: contains all plant projects retrivied from PRIDE
        pride_plants_mztab: contains all plant projects with mztab retrivied from PRIDE
        specie: specie's name to search
    """
    projetcs_total = [projects for projects,values in dict_plants.items() if f"{specie}" in values["name"]]
    projects = list(set(dict_mztab.keys()) & set(projetcs_total))
    return projects

In [12]:
def download_projects(project_list):
    """
        Download all mztab files as well as creates a directory with all files available for download
        Parameters:
            project_list: list of projects to download
    """
    for projects in project_list: 
        pride_obj = ppx.PrideProject(
            projects,
            local = os.path.join(os.getcwd(),mztab_storage_folder)
            )
        pride_obj.download(pride_plants_mztab[projects])


In [13]:
def unzip_mztabs(path):
    """
    Unzip all mztba.gz files
    """
    os.system(f"gunzip {path}")

In [14]:
def get_allmztab(path):
    """
    Get a list with all mztab files 
    """
    all_files = glob.glob(path)
    return all_files

In [15]:
# usar accession de PSM para pegar description em PRT
def PSM2PRT (mztab,accession):
    """
    Uses mztab["PRT"] accession to get pep sequence description availabel only on mztab["PSM"]
    Parameters:
        mztab: file to search
        accession: accession number
    """   
    for pst_data in mztab["PRT"].itertuples():
            if pst_data.accession == accession:
                return pst_data.description


In [24]:
def resume_mztab(file_list:list,specie ):
    """
        Returns a fasta file, per project, with all peps on all mztabs
        Parameters:
            file_list: list containg mztab files
            specie: specie to name the returning fasta file   
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        projetc_number  = mztab_obj.metadata["mzTab-ID"]
        with open(f"{specie}_{projetc_number}.fasta","w+") as file_fasta:
            for info in mztab_obj["PSM"].itertuples():
                file_fasta.write(f">{info.accession} | {PSM2PRT(mztab_obj,info.accession)} \n") # Se possível, preciso melhorar isso. O(n²) é lento
                file_fasta.write(f"{info.sequence}\n") 
    return file_fasta

#### 3.2. Caminho 2

In [16]:
def get_swissprot_sequences(accession:list,out_file_name:str,project):
    """
    convert pride accession number to swissprot accession number then downloads complete protein sequences
        Parameters:
            accession: List containg PRIDE IDs

    """
    databases_url = 'https://www.uniprot.org/uploadlists/'
    query = None
    if len(accession) == 1:
        query = accession
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{query}'}
    else:
        params = {
        'from': 'ACC+ID', #pride accession
        'to': 'SWISSPROT', #swissprot accession
        'format': 'list',
        'query': f'{" ".join(accession)}'}
    pride_accession = urllib.parse.urlencode(params)
    pride_accession = pride_accession.encode('utf-8')
    req = urllib.request.Request(databases_url, pride_accession)
    with urllib.request.urlopen(req) as accession_file:
        response = accession_file.read()
    swissprot_accession = list(response.decode('utf-8').split("\n"))

    
    # Get protein sequences from swissprot
    swissprot_url = "http://www.uniprot.org/uniprot/"
    with open(f"{out_file_name}_{project}.complete.fasta","w+") as fasta:
        for IDs in swissprot_accession:
            joint_url = swissprot_url + IDs + ".fasta"
            swissprot_response = requests.post(joint_url)
            raw_data = "".join(swissprot_response.text)
            Seq = StringIO(raw_data)
            for seq_info in SeqIO.parse(Seq,'fasta'):
                fasta.write(f">{seq_info.description}\n")
                fasta.write(f"{seq_info.seq}\n")
    return fasta

In [17]:
def concatenate_swissprot(file_list, specie):
    """
    Concatenate get_swissprot_sequences results
    """
    for files in file_list:
        mztab_obj = mztab.MzTab(files)
        project_number  = mztab_obj.metadata["mzTab-ID"]
        accession_list = mztab_obj["PRT"].accession.values
        get_swissprot_sequences(accession_list,specie,project_number)

## *Cafea arabica*

In [18]:
mztab_storage_folder = "C.arabica"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/C.arabica


Primeiro precisamos saber quantos projetos existem para cada espécie que desejamos analisar

In [19]:
specie = "Coffea arabica"
projects = get_projects(specie = specie)
projects

['PXD002963']

Com os ID dos podemos baixar os arquivos mztab de cada projeto.

Pode levar algum tempo em caso de espécies com muitos projetos

In [20]:
download_projects(projects)

TOTAL:   0%|          | 0/1 [00:00<?, ?files/s]

generated/2experimentos.pride.mztab.gz:   0%|          | 0.00/846k [00:00<?, ?b/s]

Como queremos todas as sequências peptídicas, precisamos agrupas a informação de todos os mztabs

In [21]:
#Unzip mztab.gz
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder,"generated/*.gz"))

gzip: /home/tiago/documents/lncRNA/C.arabica/generated/2experimentos.pride.mztab already exists;	not overwritten


In [22]:
all_mztab_files = get_allmztab(os.path.join(os.getcwd(),mztab_storage_folder,"generated/*.mztab"))
all_mztab_files

['/home/tiago/documents/lncRNA/C.arabica/generated/2experimentos.pride.mztab']

Como queremos todas as sequências peptídicas, precisamos agrupas a informação de todos os mztabs

In [ ]:
resume_mztab(all_mztab_files,specie)

Se quisermos usar as proteínas completas temos que converter os ID do PRIDE para o Swissprot

In [ ]:
concatenate_swissprot(all_mztab_files,specie)

### BLAST

[Genoma de ref para blast. Versão 1](https://www.ncbi.nlm.nih.gov/assembly/GCF_003713225.1#/def)

In [ ]:
#Testar o blast para proteínas não caracterizadas
from Bio import SeqIO
with open("Caffea_arabica_unchart_prot_PXD002963.fasta","w+") as file:
  for prots in SeqIO.parse("/home/tiago/documents/lncRNA/Coffea_arabica.PXD002963-58016.complete.fasta","fasta"):
    if "Uncharacterized protein" in prots.description:
      file.write(f">{prots.id}\n")
      file.write(f"{prots.seq}\n")
   

Criar bancos de dados para blast local

Database com o genoma

In [ ]:
!makeblastdb -in /home/tiago/documents/lncRNA/blast_databases/sources/GCF_003713225.1_Cara_1.0_genomic.fna -out /home/tiago/documents/lncRNA/blast_databases/C.arabica/genomic/Carabica.genomic -dbtype nucl

BLAST no genoma

In [ ]:
!tblastn -db /home/tiago/documents/lncRNA/blast_databases/C.arabica/genomic/Carabica.genomic -query /home/tiago/documents/lncRNA/Caffea_arabica_unchart_prot_PXD002963.fasta -out /home/tiago/documents/lncRNA/c.arabica.genomic.csv -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qseq sseq" -evalue 1e-10

In [26]:
col_name = ["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore","qseq","sseq"]
carabica_df_unchart_prot = pd.read_csv(
    "/home/tiago/documents/lncRNA/c.arabica.genomic.csv",
    sep = "\t",
    header = 0
    )
carabica_df_unchart_prot.columns = col_name

- 1.  qseqid: query or source (e.g., gene) sequence id

- 2.  sseqid: subject  or target (e.g., reference genome) sequence id

- 3.  pident: percentage of identical matches

- 4.  length: alignment length (sequence overlap)

- 5.  mismatch :number of mismatches

- 6.  gapopen: number of gap openings

- 7.  qstart: start of alignment in query

- 8.  qend: end of alignment in query

- 9.  sstart: start of alignment in subject

- 10. send: end of alignment in subject

- 11. evalue: expect value

- 12. bitscore: bit score

- 13. qseq: Aligned part of query sequence

- 14. sseq:      Aligned part of subject sequence

In [27]:
carabica_df_unchart_prot.head(5)

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,qseq,sseq
0,tr|A0A068UBP4|A0A068UBP4_COFCA,NC_039901.1,33.477,463,297,7,94,550,20754541,20755914,6.500000e-73,261.0,LPEPVSALSGNRLNLHNRILKLIRENDLEEAALYTRHSIYSNCKPT...,MPDSTSALTGPRLNLHNRVQSLIRAGDLDSASAIARHAVFSNVRPT...
1,tr|A0A068UBP4|A0A068UBP4_COFCA,NC_039901.1,27.907,387,269,4,115,501,15374927,15376057,3.250000e-37,154.0,LIRENDLEEAALYTRHSIYSNCKPTIYTCNAVMAAQLRQARYADLL...,LCKEGKLKRAKEFIRQMEGFGFKPNVVTYNTVIHGYCLKGDLEEAN...
2,tr|A0A068UBP4|A0A068UBP4_COFCA,NC_039901.1,25.862,406,278,6,136,526,66567255,66566062,6.540000e-36,150.0,CKPTIYTCNAVMAAQLRQARYADLLSLHRFITQAG-VAANIVTYNL...,CEPNGVMFGAVINGLCKVGNTGKAVELLRVMVKEGRVKPNVIIYST...
3,tr|A0A068UBP4|A0A068UBP4_COFCA,NC_039901.1,26.018,442,276,8,112,513,67045344,67046636,6.510000e-33,141.0,ILKLIRENDLEEAALYTRHSIYSNCKPTIYTCNAVMAAQLRQARYA...,ILALVNNKKLDLAVDGFRRAGDYGFKLSVISCNPMLGELVKNGKVG...
4,tr|A0A068UBP4|A0A068UBP4_COFCA,NC_039901.1,23.928,443,291,6,123,529,7099562,7100860,1.620000e-32,140.0,EAALYTRHSIYSNCKPTIYTCNAVMAAQLRQARYADLLSLHRFITQ...,EKAVRVMHILEQYGEPDVFVYNAVISGFCKVNKTNSANEVLNRMRV...


In [28]:
Carabica_profile = ProfileReport(carabica_df_unchart_prot, minimal = True)

Podemos usar profiling para filtrar selecionar parâmetro e filtrar sequências de melhor qualidade

In [29]:
Carabica_profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
Carabica_profile.to_file("Caffea_arabica.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [53]:
from Bio.Seq import Seq
with open("C.arabica.candidate_seq.alig.fasta","w+") as file:
    for info in carabica_df_unchart_prot.itertuples():
        file.write(f">{info.sseqid}\n")
        remove_gaps = str(Seq(info.sseq).ungap("-")) #Retira gaps "-"
        file.write(f"{remove_gaps}\n")
print("O arquivo C.arabica.candidate_seq.alig.fasta contêm sequências candidatas")

O arquivo C.arabica.candidate_seq.alig.fasta contêm sequências candidatas


## *Arabidopsis thaliana*

In [48]:
specie = "Arabidopsis thaliana"
projects = get_projects(specie = specie)
projects

['PXD000869',
 'PRD000349',
 'PXD000567',
 'PXD005168',
 'PXD006352',
 'PXD007979',
 'PXD001179',
 'PRD000096',
 'PXD001879',
 'PXD002396',
 'PXD000235',
 'PXD003352',
 'PXD004027',
 'PXD000692',
 'PXD009918',
 'PRD000581',
 'PXD001562',
 'PXD000136',
 'PXD000211',
 'PXD000880',
 'PXD001880',
 'PRD000508',
 'PXD005408',
 'PRD000084',
 'PRD000635',
 'PRD000314',
 'PXD001252',
 'PXD009392',
 'PXD001883',
 'PXD003923',
 'PRD000634',
 'PXD004896',
 'PXD000521',
 'PXD013264',
 'PXD006807',
 'PXD003449',
 'PXD000417',
 'PXD000566',
 'PXD001177',
 'PXD006328',
 'PRD000664',
 'PXD000280',
 'PXD000223',
 'PRD000141',
 'PXD001286',
 'PRD000246',
 'PXD002606',
 'PXD006848',
 'PXD001881',
 'PXD001885',
 'PXD013999',
 'PXD007678',
 'PXD004267',
 'PXD005179',
 'PXD009510',
 'PXD000908',
 'PXD006880',
 'PXD001877',
 'PXD001884',
 'PXD002075',
 'PXD001878',
 'PRD000737',
 'PXD000515',
 'PXD005740',
 'PXD001119',
 'PRD000392',
 'PRD000722',
 'PXD000693',
 'PXD004025',
 'PRD000289',
 'PXD003103',
 'PXD0

In [49]:
mztab_storage_folder = "A.thaliana"
try:
    os.makedirs(os.path.join(os.getcwd(),mztab_storage_folder))
except:
    print(f"Folder already exist: {os.path.join(os.getcwd(),mztab_storage_folder)}")

Folder already exist: /home/tiago/documents/lncRNA/A.thaliana


In [50]:
download_projects(['PRD000314']) # Projeto de exemplo

TOTAL:   0%|          | 0/6 [00:00<?, ?files/s]

generated/PRIDE_Exp_Complete_Ac_14834.pride.mztab.gz:   0%|          | 0.00/3.30M [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_14835.pride.mztab.gz:   0%|          | 0.00/3.13M [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_14836.pride.mztab.gz:   0%|          | 0.00/1.50M [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_14837.pride.mztab.gz:   0%|          | 0.00/1.55M [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_14838.pride.mztab.gz:   0%|          | 0.00/1.27M [00:00<?, ?b/s]

generated/PRIDE_Exp_Complete_Ac_14839.pride.mztab.gz:   0%|          | 0.00/1.88M [00:00<?, ?b/s]

In [27]:
unzip_mztabs(path = os.path.join(os.getcwd(),mztab_storage_folder,"generated/*.gz"))

In [51]:
all_mztab_files = get_allmztab(os.path.join(os.getcwd(),mztab_storage_folder,"generated/*.mztab"))
all_mztab_files

['/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14836.pride.mztab',
 '/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14839.pride.mztab',
 '/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14838.pride.mztab',
 '/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14834.pride.mztab',
 '/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14835.pride.mztab',
 '/home/tiago/documents/lncRNA/A.thaliana/generated/PRIDE_Exp_Complete_Ac_14837.pride.mztab']

In [29]:
resume_mztab(all_mztab_files,specie)

<_io.TextIOWrapper name='Arabidopsis thaliana_PRD000314-14837.fasta' mode='w+' encoding='UTF-8'>

Os IDS dos projetos de *A. thaliana* estão diferentes dos de *C. arabica*. Por isso, a função de converter os IDs não funcionou

In [ ]:
concatenate_swissprot(all_mztab_files,specie)